In [1]:
import json
import os
import unittest

# Archivos json
HOTELS_FILE = "hotels.json"
CUSTOMERS_FILE = "customers.json"
RESERVATIONS_FILE = "reservations.json"

#  Función para cargar y salvar datos.
def load_data(file):
    if os.path.exists(file):
        try:
            with open(file, "r") as f:
                return json.load(f)
        except json.JSONDecodeError:
            print(f"Error: Invalid data in {file}")
            return {}
    return {}

def save_data(file, data):
    with open(file, "w") as f:
        json.dump(data, f, indent=4)

#  class hotel
class Hotel:
    def __init__(self, name, location, rooms):
        self.name = name
        self.location = location
        self.rooms = rooms
        self.available_rooms = rooms

    def to_dict(self):
        return {"name": self.name, "location": self.location, "rooms": self.rooms, "available_rooms": self.available_rooms}

    @staticmethod
    def create_hotel(name, location, rooms):
        hotels = load_data(HOTELS_FILE)
        hotels[name] = {"location": location, "rooms": rooms, "available_rooms": rooms}
        save_data(HOTELS_FILE, hotels)
        print(f"Hotel {name} created.")

    @staticmethod
    def delete_hotel(name):
        hotels = load_data(HOTELS_FILE)
        if name in hotels:
            del hotels[name]
            save_data(HOTELS_FILE, hotels)
            print(f"Hotel {name} deleted.")
        else:
            print("Hotel not found.")

    @staticmethod
    def display_hotels():
        hotels = load_data(HOTELS_FILE)
        return hotels

# Customer Class
class Customer:
    def __init__(self, name, email):
        self.name = name
        self.email = email

    def to_dict(self):
        return {"name": self.name, "email": self.email}

    @staticmethod
    def create_customer(name, email):
        customers = load_data(CUSTOMERS_FILE)
        customers[name] = {"email": email}
        save_data(CUSTOMERS_FILE, customers)
        print(f"Customer {name} created.")

    @staticmethod
    def delete_customer(name):
        customers = load_data(CUSTOMERS_FILE)
        if name in customers:
            del customers[name]
            save_data(CUSTOMERS_FILE, customers)
            print(f"Customer {name} deleted.")
        else:
            print("Customer not found.")

    @staticmethod
    def display_customers():
        return load_data(CUSTOMERS_FILE)

# Reservation Class
class Reservation:
    @staticmethod
    def create_reservation(customer_name, hotel_name):
        hotels = load_data(HOTELS_FILE)
        customers = load_data(CUSTOMERS_FILE)
        reservations = load_data(RESERVATIONS_FILE)

        if customer_name not in customers or hotel_name not in hotels:
            print("Invalid customer or hotel.")
            return

        if hotels[hotel_name]["available_rooms"] > 0:
            reservations[customer_name] = hotel_name
            hotels[hotel_name]["available_rooms"] -= 1
            save_data(HOTELS_FILE, hotels)
            save_data(RESERVATIONS_FILE, reservations)
            print(f"Reservation for {customer_name} at {hotel_name} created.")
        else:
            print("No available rooms.")

    @staticmethod
    def cancel_reservation(customer_name):
        reservations = load_data(RESERVATIONS_FILE)
        hotels = load_data(HOTELS_FILE)

        if customer_name in reservations:
            hotel_name = reservations[customer_name]
            del reservations[customer_name]
            hotels[hotel_name]["available_rooms"] += 1
            save_data(HOTELS_FILE, hotels)
            save_data(RESERVATIONS_FILE, reservations)
            print(f"Reservation for {customer_name} canceled.")
        else:
            print("No reservation found.")

# Unit Tests
class TestReservationSystem(unittest.TestCase):
    def setUp(self):
        save_data(HOTELS_FILE, {})
        save_data(CUSTOMERS_FILE, {})
        save_data(RESERVATIONS_FILE, {})

    def test_create_hotel(self):
        Hotel.create_hotel("Grand Hotel", "NY", 10)
        Hotel.create_hotel("Marriot", "MX", 10)
        hotels = load_data(HOTELS_FILE)
        self.assertIn("Grand Hotel", hotels)
        self.assertIn("Marriot", hotels)

    def test_create_customer(self):
        Customer.create_customer("Diana", "Diana@example.com")
        Customer.create_customer("Diego", "diego@example.com")
        customers = load_data(CUSTOMERS_FILE)
        self.assertIn("Diana", customers)
        self.assertIn("Diego", customers)

    def test_create_reservation(self):
        Hotel.create_hotel("Grand Hotel", "NY", 10)
        Customer.create_customer("Diana", "Diana@example.com")
        Reservation.create_reservation("Diana", "Grand Hotel")
        reservations = load_data(RESERVATIONS_FILE)
        self.assertIn("Diana", reservations)

        Hotel.create_hotel("Marriot", "MX", 10)
        Customer.create_customer("Diego", "diego@example.com")
        Reservation.create_reservation("Diego", "Marriot")
        reservations = load_data(RESERVATIONS_FILE)
        self.assertIn("Diego", reservations)

    def test_cancel_reservation(self):
        Hotel.create_hotel("Grand Hotel", "NY", 10)
        Customer.create_customer("Diana", "diana@example.com")
        Reservation.create_reservation("Diana", "Grand Hotel")
        Reservation.cancel_reservation("Diana")
        reservations = load_data(RESERVATIONS_FILE)
        self.assertNotIn("Diana", reservations)

        Hotel.create_hotel("Marriot", "MX", 10)
        Customer.create_customer("Diego", "diego@example.com")
        Reservation.create_reservation("Diego", "Marriot")
        Reservation.cancel_reservation("Diego")
        reservations = load_data(RESERVATIONS_FILE)
        self.assertNotIn("Diego", reservations)
if __name__ == "__main__":
    unittest.main(argv=[''], verbosity=2, exit=False)

test_cancel_reservation (__main__.TestReservationSystem.test_cancel_reservation) ... ok
test_create_customer (__main__.TestReservationSystem.test_create_customer) ... ok
test_create_hotel (__main__.TestReservationSystem.test_create_hotel) ... ok
test_create_reservation (__main__.TestReservationSystem.test_create_reservation) ... ok

----------------------------------------------------------------------
Ran 4 tests in 0.080s

OK


Hotel Grand Hotel created.
Customer Diana created.
Reservation for Diana at Grand Hotel created.
Reservation for Diana canceled.
Hotel Marriot created.
Customer Diego created.
Reservation for Diego at Marriot created.
Reservation for Diego canceled.
Customer Diana created.
Customer Diego created.
Hotel Grand Hotel created.
Hotel Marriot created.
Hotel Grand Hotel created.
Customer Diana created.
Reservation for Diana at Grand Hotel created.
Hotel Marriot created.
Customer Diego created.
Reservation for Diego at Marriot created.
